# Xception

In [1]:
import densenet

from keras.optimizers import RMSprop
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.callbacks import TensorBoard
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

Using TensorFlow backend.


In [2]:
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/'
train_data_dir = DATA_HOME_DIR+'train'

In [3]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd

from sklearn.cross_validation import train_test_split
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss

/home/paperspace/anaconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def split_validation_set(train, target, test_size):
    random_state = 51
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [5]:
def split_validation_set_with_hold_out(train, target, test_size):
    random_state = 51
    train, X_test, target, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    X_train, X_holdout, y_train, y_holdout = train_test_split(train, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, X_holdout, y_train, y_test, y_holdout

# Another approach to loading the data

In [6]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import glob
import cv2

In [7]:
def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (32, 32), cv2.INTER_LINEAR) #use cv2.resize(img, (64, 64), cv2.INTER_LINEAR)
    return [path, resized]

InceptionV3 hat pre_process input: https://github.com/fchollet/deep-learning-models/blob/master/inception_v3.py
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [8]:
def normalize_image_features(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
    fdata = fdata.astype('float32')
    return fdata

In [9]:
train = glob.glob('data/train/**/*.jpg')# + glob.glob('../input/additional/**/*.jpg')

In [10]:
train[1]

'data/train/Type_3/1127.jpg'

In [11]:
train[1].split('/')

['data', 'train', 'Type_3', '1127.jpg']

In [12]:
train = pd.DataFrame([[p.split('/')[2],p.split('/')[3],p] for p in train], columns = ['type','image','path'])#[::5] #limit for Kaggle Demo

In [13]:
train = im_stats(train)
train = train[train['size'] != '0 0'].reset_index(drop=True) #remove bad images
print('dropped bad images')
train_data = normalize_image_features(train['path'])
np.save('train.npy', train_data, allow_pickle=True, fix_imports=True)
print('Finished loading data!')

dropped bad images
Finished loading data!


In [14]:
le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
train_target = np_utils.to_categorical(train_target, num_classes=3)

In [15]:
print(train_target[:5])

[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


In [16]:
print(le.classes_) #in case not 1 to 3 order
np.save('train_target.npy', train_target, allow_pickle=True, fix_imports=True)

['Type_1' 'Type_2' 'Type_3']


Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (1480, 3, 32, 32) (32 channels).

In [17]:
print(train_data.shape)

(1480, 32, 32, 3)


In [18]:
train_data = preprocess_input(train_data)

## Splitting the data

In [19]:
X_train, X_val, Y_train, Y_val = split_validation_set(train_data, train_target, 0.3)

In [20]:
print(X_train.shape)

(1036, 32, 32, 3)


# Now building the model

In [21]:
def printTrainableLayers(model):
    for i, layer in enumerate(model.layers):
       if layer.trainable == True: 
           print(i, layer.name)

In [22]:
def printAllLayers(model):
    for i, layer in enumerate(model.layers):
       print(i, layer.name)

In [23]:
base_model = InceptionV3(weights='imagenet', include_top=False)
print('Finished Loading')

Finished Loading


In [24]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
# and a logistic layer -- let's say we have 200 classes
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax', kernel_initializer='glorot_uniform')(x)

In [25]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [26]:
printAllLayers(model)

(0, 'input_1')
(1, 'conv2d_1')
(2, 'batch_normalization_1')
(3, 'activation_1')
(4, 'conv2d_2')
(5, 'batch_normalization_2')
(6, 'activation_2')
(7, 'conv2d_3')
(8, 'batch_normalization_3')
(9, 'activation_3')
(10, 'max_pooling2d_1')
(11, 'conv2d_4')
(12, 'batch_normalization_4')
(13, 'activation_4')
(14, 'conv2d_5')
(15, 'batch_normalization_5')
(16, 'activation_5')
(17, 'max_pooling2d_2')
(18, 'conv2d_9')
(19, 'batch_normalization_9')
(20, 'activation_9')
(21, 'conv2d_7')
(22, 'conv2d_10')
(23, 'batch_normalization_7')
(24, 'batch_normalization_10')
(25, 'activation_7')
(26, 'activation_10')
(27, 'average_pooling2d_1')
(28, 'conv2d_6')
(29, 'conv2d_8')
(30, 'conv2d_11')
(31, 'conv2d_12')
(32, 'batch_normalization_6')
(33, 'batch_normalization_8')
(34, 'batch_normalization_11')
(35, 'batch_normalization_12')
(36, 'activation_6')
(37, 'activation_8')
(38, 'activation_11')
(39, 'activation_12')
(40, 'mixed0')
(41, 'conv2d_16')
(42, 'batch_normalization_16')
(43, 'activation_16')
(44, 'c

In [27]:
# set the first 311 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:311]:
    layer.trainable = False

In [28]:
printTrainableLayers(model)

(311, 'global_average_pooling2d_1')
(312, 'dense_1')
(313, 'dropout_1')
(314, 'dense_2')
(315, 'dropout_2')
(316, 'dense_3')


In [29]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

batch_normalization_75 (BatchNor (None, None, None, 19 576         conv2d_75[0][0]                  
____________________________________________________________________________________________________
activation_71 (Activation)       (None, None, None, 19 0           batch_normalization_71[0][0]     
____________________________________________________________________________________________________
activation_75 (Activation)       (None, None, None, 19 0           batch_normalization_75[0][0]     
____________________________________________________________________________________________________
conv2d_72 (Conv2D)               (None, None, None, 32 552960      activation_71[0][0]              
____________________________________________________________________________________________________
conv2d_76 (Conv2D)               (None, None, None, 19 331776      activation_75[0][0]              
___________________________________________________________________________________________

In [30]:
filepath="Xception-top-improvement_learning_rate_decy_0.01-{epoch:02d}-{val_loss:}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
tbCallBack = TensorBoard(log_dir='./graph_xcep_top', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list.append(tbCallBack)

In [31]:
opt = RMSprop(lr=1e-2, rho=0.9, decay=0.01)
#opt = SGD(lr=1e-2, momentum=0.9, decay=0.01, nesterov=True)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [32]:
batch_size = 16
nb_classes = 3
nb_epoch = 300
nb_train_samples = X_train.shape[0]
nb_val_samples = X_val.shape[0]

In [33]:
#rescale=1./255, removed because of pre_process
datagen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest')
datagen.fit(train_data)

In [34]:
train_generator = datagen.flow(X_train,Y_train)

Sample hat shape: (32,32,32,3)

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=nb_epoch,
        validation_data=(X_val, Y_val),
        validation_steps=nb_val_samples // batch_size,
        callbacks=callbacks_list)

Epoch 1/300


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("xcep_top_only.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("xcep_top_only.h5")
print("Saved model to disk")

# Submit to kaggle for testing purposes

## Loading test data

In [ ]:
test = glob.glob('data/test/**/*.jpg')
test = pd.DataFrame([[p.split('/')[3],p] for p in test], columns = ['image','path']) #[::20] #limit for Kaggle Demo
test_data = normalize_image_features(test['path'])
np.save('test.npy', test_data, allow_pickle=True, fix_imports=True)

test_id = test.image.values
np.save('test_id.npy', test_id, allow_pickle=True, fix_imports=True)

In [ ]:
print(test_data.shape)

In [ ]:
test_data_channels_last = np.transpose(test_data, (0, 2, 3, 1))

In [ ]:
print(test_data_channels_last.shape)

In [ ]:
# Calculate class posteriors probabilities
y_probabilities = model.predict(test_data_channels_last, batch_size=16, verbose=0)
print(len(y_probabilities))

In [ ]:
print(y_probabilities[:5])

In [ ]:
df = pd.DataFrame(y_probabilities, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission.csv', index=False)

In [ ]:
%ls